> pip install hypetopt

https://www.analyticsvidhya.com/blog/2020/09/alternative-hyperparameter-optimization-technique-you-need-to-know-hyperopt/

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK

In [ ]:
import seaborn as sns

In [ ]:
v_typ = []
v_x = []
i = 0

def funkcja(args):
    global v_typ, v_x, i

    typ = args['type']
    x = args['x']


    v_typ.append(typ)
    v_x.append(x)
    i += 1

    if typ == 'typ A':
        y = args['y']
        return {'loss': x * y , 'status': STATUS_OK }
    else:
        return {'loss': x ** 2, 'status': STATUS_OK }

In [ ]:
space = hp.choice('class_type',
[
    {
        'type': 'typ A',
        'x': hp.uniform('x_uniform', 0, 10),
        'y': hp.normal('x_normal', 0, 10)
    },
    {
        'type': 'typ B',
        'x': hp.normal('x2_normal', 0, 10)
    }
])

In [ ]:
best = fmin(funkcja, space, algo=tpe.suggest, max_evals=100)

In [ ]:
best

In [ ]:
sns.scatterplot(x=range(0, i), y=v_x, hue=v_typ)

In [ ]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return {
        'loss': x ** 2 + 10,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time': time.time(),
        'other_stuff': {'type': None, 'value': [0, 1, 2]},
        # -- attachments are handled differently
        'attachments':
            {'time_module': pickle.dumps(time.time)}
        }
trials = Trials()
best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100,
    trials=trials)

print(best)

In [ ]:
best


# -------

In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.datasets import load_iris

In [ ]:
X = load_iris()['data']
y = load_iris()['target']

iris_df = pd.DataFrame(X, columns=['sl', 'sw', 'pl', 'pw'])
iris_df['species'] = y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_df.drop('species', axis=1),
                                                    iris_df['species'],
                                                    test_size=0.2)

In [ ]:
# lista kolumn numerycznych
cols_numerical = X_train.select_dtypes(include=['int64', 'float64']).columns

# transformer dla kolumn numerycznych
transformer_numerical = Pipeline(steps =[
    ('first_scaler', StandardScaler()),
    ('dim_red', PCA()),
    ('second_scaler', StandardScaler())
])

In [ ]:
# analogicznie dla kolumn categorycznych jeśli trzeba

In [ ]:
# preprocesor danych
preprocessor = ColumnTransformer(transformers=[
    ('numerical', transformer_numerical, cols_numerical)
])

In [ ]:
# pipeline uniwersalny
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', None)])
# klasyfikatory                            
classifiers = [
    RandomForestClassifier(),
    SVC(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    XGBoost(),
    CatBoost(),
    LightGBM()
]


In [ ]:
list(svc_pipeline.get_params().keys())

In [ ]:
list(rfc_pipeline.get_params().keys())

In [ ]:
for model in (svc_pipeline, rfc_pipeline):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(acc)

In [ ]:
svc_pipeline.get_params()
